<a href="https://colab.research.google.com/github/HernanMagallanes/TensorFlow_course/blob/main/Module_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 6: Natural Language Processing with RNNs 
Natural Language Processing (or NLP for short) is a discipline in computing that deals with the communication between natural (human) languages and computer languages. A common example of NLP is something like spellcheck or autocomplete. Essentially NLP is the field that focuses on how computers can understand and/or process natural/human languages. 

###Recurrent Neural Networks

In this tutorial we will introduce a new kind of neural network that is much more capable of processing sequential data such as text or characters called a **recurrent neural network** (RNN for short). 

We will learn how to use a reccurent neural network to do the following:
- Sentiment Analysis
- Character Generation 

RNN's are complex and come in many different forms so in this tutorial we wil focus on how they work and the kind of problems they are best suited for.



## Sequence Data

Since textual data contains many words that follow in a very specific and meaningful order, we need to be able to keep track of each word and when it occurs in the data. Simply encoding say an entire paragraph of text into one data point wouldn't give us a very meaningful picture of the data and would be very difficult to do anything with. This is why we treat text as a sequence and process one word at a time. We will keep track of where each of these words appear and use that information to try to understand the meaning of peices of text.


##Encoding Text
As we know machine learning models and neural networks don't take raw text data as an input. This means we must somehow encode our textual data to numeric values that our models can understand. There are many different ways of doing this and we will look at a few examples below. 

Before we get into the different encoding/preprocessing methods let's understand the information we can get from textual data by looking at the following two movie reviews.

```I thought the movie was going to be bad, but it was actually amazing!```

```I thought the movie was going to be amazing, but it was actually bad!```

Although these two setences are very similar we know that they have very different meanings. This is because of the **ordering** of words, a very important property of textual data.

Now keep that in mind while we consider some different ways of encoding our textual data.

###Bag of Words
The first and simplest way to encode our data is to use something called **bag of words**. This is a pretty easy technique where each word in a sentence is encoded with an integer and thrown into a collection that does not maintain the order of the words but does keep track of the frequency. Have a look at the python function below that encodes a string of text into bag of words. 

In [ ]:
# maps word to integer representing it
vocab = {}  

word_encoding = 1

def bag_of_words(text):
  global word_encoding

  # create a list of all of the words in the text, well assume there is no grammar in our text for this example
  words = text.lower().split(" ")  

  # stores all of the encodings and their frequency
  bag = {}

  for word in words:
    if word in vocab:
      # get encoding from vocab
      encoding = vocab[word]  

    else:
      vocab[word] = word_encoding
      encoding = word_encoding
      word_encoding += 1
    
    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1
  
  return bag

text = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(text)

print(bag)
print(vocab)

{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [ ]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_bag = bag_of_words(positive_review)
neg_bag = bag_of_words(negative_review)

print("Positive:", pos_bag)
print("Negative:", neg_bag)

Positive: {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1}
Negative: {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 21: 1, 18: 1, 19: 1, 20: 1, 17: 1}


In [ ]:
# Integer Encoding

vocab = {}  
word_encoding = 1

def one_hot_encoding(text):
  global word_encoding

  words = text.lower().split(" ") 
  encoding = []  

  for word in words:
    if word in vocab:
      code = vocab[word]  
      encoding.append(code) 
    else:
      vocab[word] = word_encoding
      encoding.append(word_encoding)
      word_encoding += 1
  
  return encoding

text = "this is a test to see if this test will work is is test a a"
encoding = one_hot_encoding(text)
print(encoding)
print(vocab)

[1, 2, 3, 4, 5, 6, 7, 1, 4, 8, 9, 2, 2, 4, 3, 3]
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [ ]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_encode = one_hot_encoding(positive_review)
neg_encode = one_hot_encoding(negative_review)

print("Positive:", pos_encode)
print("Negative:", neg_encode)

Positive: [10, 11, 12, 13, 14, 15, 5, 16, 17, 18, 19, 14, 20, 21]
Negative: [10, 11, 12, 13, 14, 15, 5, 16, 21, 18, 19, 14, 20, 17]


Much better, now we are keeping track of the order of words and we can tell where each occurs. But this still has a few issues with it. Ideally when we encode words, we would like similar words to have similar labels and different words to have very different labels. For example, the words happy and joyful should probably have very similar labels so we can determine that they are similar. While words like horrible and amazing should probably have very different labels. The method we looked at above won't be able to do something like this for us. This could mean that the model will have a very difficult time determing if two words are similar or not which could result in some pretty drastic performace impacts.


###Word Embeddings
Luckily there is a third method that is far superior, **word embeddings**. This method keeps the order of words intact as well as encodes similar words with very similar labels. It attempts to not only encode the frequency and order of words but the meaning of those words in the sentence. It encodes each word as a dense vector that represents its context in the sentence.

Unlike the previous techniques word embeddings are learned by looking at many different training examples. You can add what's called an *embedding layer* to the beggining of your model and while your model trains your embedding layer will learn the correct embeddings for words. You can also use pretrained embedding layers.

Classify or translate every single one of our words into a vector (n dims)
every single component of that vector will kind of tell us what group it belongs to, or how similar it is to other word.


This is the technique we will use for our examples and its implementation will be showed later on.



##Recurrent Neural Networks (RNN's)
Now that we've learned a little bit about how we can encode text it's time to dive into recurrent neural networks. Up until this point we have been using something called **feed-forward** neural networks. This simply means that all our data is fed forwards (all at once) from left to right through the network. This was fine for the problems we considered before but won't work very well for processing text. After all, even we (humans) don't process text all at once. We read word by word from left to right and keep track of the current meaning of the sentence so we can understand the meaning of the next word. Well this is exaclty what a recurrent neural network is designed to do. When we say recurrent neural network all we really mean is a network that contains a loop. 

A RNN will process one word at a time while maintaining an internal memory of what it's already seen. This will allow it to treat words differently based on their order in a sentence and to slowly build an understanding of the entire input, one word at a time.

This is why we are treating our text data as a sequence! So that we can pass one word at a time to the RNN.

##LSTM
The layer we dicussed in depth above was called a *simpleRNN*. However, there does exist some other recurrent layers (layers that contain a loop) that work much better than a simple RNN layer. The one we will talk about here is called LSTM (Long Short-Term Memory). This layer works very similarily to the simpleRNN layer but adds a way to access inputs from any timestep in the past. Whereas in our simple RNN layer input from previous timestamps gradually disappeared as we got further through the input. 

With a LSTM we have a long-term memory data structure storing all the previously seen inputs as well as when we saw them. This allows for us to access any previous value we want at any point in time. This adds to the complexity of our network and allows it to discover more useful relationships between inputs and when they appear. 



##Sentiment Analysis
And now time to see a recurrent neural network in action. For this example, we are going to do something called sentiment analysis.

The formal definition of this term from Wikipedia is as follows:

*the process of computationally identifying and categorizing opinions expressed in a piece of text, especially in order to determine whether the writer's attitude towards a particular topic, product, etc. is positive, negative, or neutral.*

The example we’ll use here is classifying movie reviews as either postive, negative or neutral.


###Movie Review Dataset
Well start by loading in the IMDB movie review dataset from keras. This dataset contains 25,000 reviews from IMDB where each one is already preprocessed and has a label as either positive or negative. Each review is encoded by integers that represents how common a word is in the entire dataset. For example, a word encoded by the integer 3 means that it is the 3rd most common word in the dataset.
 


In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.
17473536/17464789 [==============================] - 1s 0us/step


In [ ]:
# review num 1 
len(train_data[1])

189

In [ ]:
# Preprocessing

# If we have a look at some of our loaded in reviews, we'll notice that they are different lengths. This is an issue.
#  We cannot pass different length data into our neural network. Therefore, we must make each review the same length. 
#  To do this we will follow the procedure below:
# - if the review is greater than 250 words then trim off the extra words
# - if the review is less than 250 words add the necessary amount of 0's to make it equal to 250.

train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)


###Creating the Model
Now it's time to create the model. We'll use a word embedding layer as the first layer in our model and add a LSTM layer afterwards that feeds into a dense node to get our predicted sentiment. 

32 stands for the output dimension of the vectors generated by the embedding layer. We can change this value if we'd like!

In [ ]:
# model

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# training
#  Runtime > Change Runtime Type > "GPU"
#  under hardware accelerato

model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 41s 56ms/step - loss: 0.4250 - acc: 0.8023 - val_loss: 0.4032 - val_acc: 0.8284
Epoch 2/10
625/625 [==============================] - 35s 55ms/step - loss: 0.2453 - acc: 0.9061 - val_loss: 0.2649 - val_acc: 0.8938
Epoch 3/10
625/625 [==============================] - 34s 54ms/step - loss: 0.1816 - acc: 0.9336 - val_loss: 0.2723 - val_acc: 0.8912
Epoch 4/10
625/625 [==============================] - 34s 55ms/step - loss: 0.1535 - acc: 0.9434 - val_loss: 0.2997 - val_acc: 0.8758
Epoch 5/10
625/625 [==============================] - 34s 55ms/step - loss: 0.1278 - acc: 0.9556 - val_loss: 0.2908 - val_acc: 0.8894
Epoch 6/10
625/625 [==============================] - 34s 55ms/step - loss: 0.1120 - acc: 0.9616 - val_loss: 0.3520 - val_acc: 0.8768
Epoch 7/10
625/625 [==============================] - 34s 55ms/step - loss: 0.0959 - acc: 0.9681 - val_loss: 0.3380 - val_acc: 0.8898
Epoch 8/10
625/625 [==============================] - 34s 54ms

In [ ]:
# result

results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 12s 16ms/step - loss: 0.4489 - acc: 0.8562
[0.44885489344596863, 0.8561599850654602]


In [ ]:
# prediction

word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

1654784/1641221 [==============================] - 0s 0us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [ ]:
# while were at it lets make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

that movie was just amazing so amazing


In [ ]:
# now time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)


[0.9086821]
[0.3868512]


##RNN Play Generator

Now time for one of the coolest examples we've seen so far. We are going to use a RNN to generate a play. We will simply show the RNN an example of something we want it to recreate and it will learn how to write a version of it on its own. We'll do this using a character predictive model that will take as input a variable length sequence and predict the next character. We can use the model many times in a row with the output from the last predicition as the input for the next call to generate a sequence.


In [159]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [160]:
# dataset

path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [161]:
#Loading your own data

# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

In [162]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [163]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



###Encoding
Since this text isn't encoded yet well need to do that ourselves. We are going to encode each unique character as a different integer.

In [164]:
vocab = sorted(set(text))

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [165]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [166]:
# numeric values to text

def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


###Creating Training Examples
Remember our task is to feed the model a sequence and have it return to us the next character. This means we need to split our text data from above into many shorter sequences that we can pass to the model as training examples. 

The training examples we will prepapre will use a *seq_length* sequence as input and a *seq_length* sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

```input: Hell | output: ello```

Our first step will be to create a stream of characters from our text data.

In [167]:
# length of sequence for a training example
seq_length = 100  
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [168]:
# Next we can use the batch method to turn this stream of characters into batches of desired length.
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [169]:
def split_input_target(chunk):
  # for the example: hello

  input_text = chunk[:-1]  # hell
  target_text = chunk[1:]  # ello
  
  return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [170]:
for x, y in dataset.take(2):
  print("\n\t ~ EXAMPLE ~ \n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))


	 ~ EXAMPLE ~ 

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 

	 ~ EXAMPLE ~ 

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [171]:
# training

BATCH_SIZE = 64

# vocab is number of unique characters
VOCAB_SIZE = len(vocab)  

EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)



###Building the Model
Now it is time to build the model. We will use an embedding layer a LSTM and one dense layer that contains a node for each unique character in our training data. The dense layer will give us a probability distribution over all nodes.

In [172]:
# model 

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):

  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (64, None, 256)           16640     
                                                                 
 lstm_10 (LSTM)              (64, None, 1024)          5246976   
                                                                 
 dense_10 (Dense)            (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


### Output

In [173]:
for input_example_batch, target_example_batch in data.take(1):

  # ask our model for a prediction on our first batch of training data (64 entries)
  example_batch_predictions = model(input_example_batch)  
  
  # print out the output shape
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [174]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch

print(len(example_batch_predictions))
print(example_batch_predictions.shape)


64
(64, 100, 65)


In [175]:
# lets examine one prediction
pred = example_batch_predictions[0]

print(len(pred))
print(pred.shape)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
(100, 65)


In [176]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]

print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 8.0854801e-04 -9.9223480e-04  4.3468415e-03  7.3629282e-03
 -1.9070716e-03  4.5736516e-03  1.5923386e-03 -3.4111880e-03
 -3.0019262e-04  7.3221083e-05  3.0423494e-03  3.8779892e-03
 -3.8668008e-03  2.7549211e-03  8.5883956e-05  1.8831962e-04
 -2.6982282e-03  6.2273666e-03  4.7648046e-03 -3.0757871e-03
 -3.0656115e-03 -5.7144719e-03  2.8680398e-03 -4.9581644e-03
 -1.9108108e-03 -1.3766845e-03  1.3612036e-03  1.1895744e-03
 -2.7334916e-03  1.9611076e-03 -1.6419669e-03 -2.4415085e-03
 -1.2445350e-03  4.5772996e-03  1.2301295e-03  4.6433499e-03
  1.5803816e-03 -4.8557410e-04 -4.1440153e-03  1.3347378e-04
  4.5530573e-03 -2.2794488e-03 -9.7159256e-04  8.7035740e-05
 -1.9358440e-03 -6.5939518e-04 -3.5665717e-03  2.8350556e-03
  5.8754510e-03 -1.8701022e-03  1.2998042e-03  1.0075944e-03
 -8.3424390e-04  9.7996159e-04 -2.3945484e-03  1.8352689e-03
  6.9716689e-04  1.1184906e-03  1.1557952e-03  1.0122600e-02
 -2.3054206e-03  2.0574469e-03  1.0478300e-02 -6.2195671e-04
  8.514588

###Creating a Loss Function
Now we are going to create our own loss function for this problem. This is because our model will output a (64, sequence_length, 65) shaped tensor that represents the probability distribution of each character at each timestep for every sequence in the batch. 

However, before we do that let's have a look at a sample input and the output from our untrained model. This is so we can understand what the model is giving us.




In [177]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

# and this is what the model predicted for training sequence 1
predicted_chars  

";BGed'bG?XtmUMYTkS:dcL,h\nWRz.UFFa\nUZaNh!ID\n'Kd?hVsOpq?vG\nvJe!dCU:SeT' TDDeJyNbGrOKqYSwaXt\nrpSKI!Vd&m"

In [178]:
# So now we need to create a loss function that can compare that output to the 
# expected output and give us some numeric value representing how close the two were. 

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [179]:
# compiling the model 

model.compile(optimizer='adam', loss=loss)

In [180]:
# creating checkpoints

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [181]:
# training
# this is taking a while go to Runtime > Change Runtime Type and choose "GPU" under hardware accelerato.**

history = model.fit(data, epochs=10, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 34s 175ms/step - loss: 2.5724
Epoch 2/10
172/172 [==============================] - 32s 174ms/step - loss: 1.8668
Epoch 3/10
172/172 [==============================] - 32s 174ms/step - loss: 1.6257
Epoch 4/10
172/172 [==============================] - 31s 173ms/step - loss: 1.4980
Epoch 5/10
172/172 [==============================] - 31s 173ms/step - loss: 1.4198
Epoch 6/10
172/172 [==============================] - 32s 173ms/step - loss: 1.3644
Epoch 7/10
172/172 [==============================] - 31s 173ms/step - loss: 1.3195
Epoch 8/10
172/172 [==============================] - 31s 173ms/step - loss: 1.2818
Epoch 9/10
172/172 [==============================] - 31s 173ms/step - loss: 1.2454
Epoch 10/10
172/172 [==============================] - 32s 174ms/step - loss: 1.2102


In [182]:
# loading the model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [183]:
# Once the model is finished training, we can find the **lastest checkpoint** that stores the models weights using the following line.

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [184]:
# We can load **any checkpoint** we want by specifying the exact file to load.

# checkpoint_num = 10
# model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
# model.build(tf.TensorShape([1, None]))

###Generating Text
Now we can use the lovely function provided by tensorflow to generate some text using any starting string we'd like.

In [185]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [186]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: romeo
romeo's one of your throne;
But fled, where they have taught you.

NORTHUMBERLAND:
Pretthar, gentle enemies! but he's Paulina.

Fourth Brow:
This is but so; his grief could not
unself-eace.

BENVOLIO:
Here's a meet to him!
Farewell: 'tis the house, by the common:
The date of love's voices.

LEONTES:
Why, there is Liknes'd servant.
You, sir, he loves me lord upon this death.

TYBRAL:
I say, not so well: no turn; I must buy speaks with thee.

PETRUCHIO:
Even, so burn'd? what with good Hartical had, will weep.
I would not by this captainlas atumpats,
But but a binter
Of my poor'st, stay; and, o' the toes the truth.

GRAMIO:
King yet ere he were ask, not content.

QUEEN ELIZABHH:
By for your honesty!

MONTAGUE:
What, all-sing knife to see her grief:
I pray thee, all the adversaries
Should be guilde
